# Cryptographic operations in Python
This Jupyter notebook demonstrates the most important cryptographic operations in Python. 

## Converting strings and bytes

The code snippet makes a conversion from a regular string to a byte string and back again.

In [2]:
# declare a string
myStr = "hello world"

# convert the string to a byte string
myBytesStr = str.encode(myStr)

# print the byte string and its type. A b should be placed in front of the string
print("A byte string:",type(myBytesStr),myBytesStr)

# convert the byte string to a string
myDecStr = myBytesStr.decode()

# print the string and its type
print("A String:", type(myDecStr),myDecStr)

# check that the original string and the decoded string are the same
assert(myStr == myDecStr)


A byte string: <class 'bytes'> b'hello world'
A String: <class 'str'> hello world


## Generating a random number

In order to generate a random number for cryptographic use, it is not wise to use the standard random generator function (in python that is random from the random class). A safe alternative is to use os.urandom, which is suitable for cryptographic use. 

In [3]:
import os

# the following function generates a random number with length of len bytes
def genrandom(len: int):
    # generate a random number, the type of the number is bytes
    rand = os.urandom(len)
    
    # return the random number.
    return rand

result = genrandom(32)
print("Random number: ",type(result),result)

# you can improve the readability, by converting to a hex string
hexResult = result.hex()
print("Hex string:",type(hexResult),hexResult)


Random number:  <class 'bytes'> b'LL\xe2\x90\xd7\xcc]o\x84\xc4\x03\x04\xf5\xba\xee\x7f\x08\xa3\x90_\xb8C\x84o\xd2Q\x1b\xc2\xe4\xfc\x90\xee'
Hex string: <class 'str'> 4c4ce290d7cc5d6f84c40304f5baee7f08a3905fb843846fd2511bc2e4fc90ee


## Create random data file

The following code snippet creates a file with random data. This function is used by other parts of this notebook.

In [4]:
import os

def createrandomfile(filename: str,len: int):
    with open(filename,"wb") as f:
        f.write(os.urandom(len))

## Calculate a SHA 256 hash of a string

In [5]:
from cryptography.hazmat.primitives import hashes

def calcHash(text: str):
    # convert the string to bytes
    bytesStr = str.encode(text)
    
    # initialize the hashing function
    digest = hashes.Hash(hashes.SHA256())
    
    # add data to the hashing function
    digest.update(bytesStr) # Bytes
    
    # finalize the hashing function and obtain the hash (digestBytes)
    digestBytes = digest.finalize() # Bytes
    
    # convert the hash (bytes string) to a hex string
    hexBytes = digestBytes.hex()
    
    # return the result
    return hexBytes

# What is the hash of nothing? 
print("If you see this specific hash: ", calcHash(""), " it is that of nothing, typically an empty file")


If you see this specific hash:  e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855  it is that of nothing, typically an empty file


## Calculate the SHA 256 hash of a file

The following program creates a file, and then reads to calculate a SHA 256 hash. Most important aspects are that the file is read in binary mode (the SHA 256 digest function requires bytes as input) and the the file is read in chunks (pieces of a defined length). Each chunk, read form the file, is added to the digest function by making a call to update. 

In [6]:
## Calculate the SHA 256 hash of a file
from cryptography.hazmat.primitives import hashes

def hashfile(filename: str, chunksize: int):
    # initialize hash function
    digest = hashes.Hash(hashes.SHA256())
    
    # open the file and add data to the hashing function
    with open(filename,"rb") as f:
        # read a chunk of size chunksize from the file
        b = f.read(chunksize)
        
        # Continue until there is nothing more to read
        while b:
            # add the value of b to the digest
            digest.update(b)
            
            # read the next chunk from file
            b = f.read(chunksize)
            
    # finalize the digest to obtain the hash
    digestBytes = digest.finalize()
    
    # convert the bytes string to a hex string
    hexBytes = digestBytes.hex()
    
    # result the result
    return hexBytes

            
# creata random file of length 1000    
createrandomfile("testrandom.dat",1000)

# calculate the hash of this file
result = hashfile("testrandom.dat",1)

# Print the hash. Each time the hash is different as the file is recreated each time
print("The hash is: ", result)


The hash is:  db2e5272f2844702383e8bfd8f1f79f4bc3a55677c5ee704a90fbe97f01e3366


## AES encryption and decryption in GCM mode

The following code performs encryption using AES in Galois Counter Mode (GCM). We declare two functions, one for encryption and one for decryption. 

Both functions are implemented to read/write from a file instead of just encrypting/decrypting a single message. 

GCM mode does not need padding, so that is not included.


In [30]:
import os

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# encrypt a file in finput and write the encrypted file to foutput
def encrypt(key, aad, finput, foutput):
    # Generate a random initialisation vector (IV)
    iv = os.urandom(16)

    # Construct an AES-GCM Cipher object with the key and the IV
    encryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv)
    ).encryptor()

    # associated_data will be authenticated but not encrypted,
    # it must also be passed in on decryption.
    # This is used to verify the integrity of the encrypted data. 
    # Feed aad into the encryptor
    # This is a specific feature of GCM 
    encryptor.authenticate_additional_data(aad)

    # open the output file, where the encrypted data is written
    # note that the file is opened in binary mode
    with open(foutput,"wb") as fo:
        
        # open the input file from where the plaintext data is read
        # note that the file is opened in binary mode
        with open(finput,"rb") as fi:
            
            # read one byte from the input file. This is not efficient, but
            # demonstrates that we can encrypt data in a stream-like fashion
            bi = fi.read(1)
        
            # while bi is not None, there is data to encrypt
            while bi:
                # feed the byte into the encryption function
                # the encryption function may or may not return data
                ciphertext = encryptor.update(bi)
        
                # if there is data, write it to the output file
                if ciphertext:
                    fo.write(ciphertext)
                
                # read the next byte
                bi = fi.read(1)

        # all data is read and the input file is closed
        
        # make the final call to finalize
        ciphertext = encryptor.finalize()
        
        # if there is ciphertext, write that to the file as wll
        if ciphertext:
            fo.write(ciphertext)
        
    # return the initialization vector and return the 
    # authentication tag, which is used for integrity verification
    return (iv, encryptor.tag)

# decrypt the file from finput into foutput
def decrypt(key, iv, aad, tag, finput, foutput):
    # Construct a Cipher object, with the key, iv, and additionally the
    # tag used for authenticating the message.
    decryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv, tag),
    ).decryptor()

    # We put associated_data back in or the tag will fail to verify
    # when we finalize the decryptor.
    decryptor.authenticate_additional_data(aad)

    # open the output file 
    with open(foutput,"wb") as fo:
        
        # open the input file
        with open(finput,"rb") as fi:
            
            # read one byte form the input
            bi = fi.read(1)
            
            # while there is a byte read from the input
            while bi:
                
                # decrypt the byte
                plaintext = decryptor.update(bi)
            
                # if the decryptor returned data, write that to the output file
                if plaintext:
                    fo.write(plaintext)
    
                # read the next byte from the input
                bi = fi.read(1)
        
        # at this point, all data from the input is read
        # call the finalize function and write the result to the output            
        plaintext = decryptor.finalize()
        if plaintext:
            fo.write(plaintext)
     

filenameInput = "randomdata.dat"
filenameEncrypted= "randomdataEncrypted.dat"
filenameDecoded = "randomdataDecoded.dat"

# creata random file of length 1000    
createrandomfile(filenameInput,1000)

# create an encryption key
key = os.urandom(16)

# generate a random string that we use for integrity verification
aad = os.urandom(256)

# encrypt filenameInput to filenameEncrypted
iv, tag = encrypt(key,aad,filenameInput,filenameEncrypted)

# decrypt filenameEncrypted into filennameDecoded
decrypt(key,iv,aad, tag,filenameEncrypted,filenameDecoded)

# Dump the hashes of the files
for filename in [filenameInput, filenameEncrypted, filenameDecoded]:
    print("The SHA526 of '{}' is '{}'".format(filename,hashfile(filename,1)))
    
print("The hashes of the plaintext and decoded file should be the same")

The SHA526 of 'randomdata.dat' is '0506f547b0f02251e5d05b77cce28bd46e2a6a5a7b2533a01dec6bc286307b0b'
The SHA526 of 'randomdataEncrypted.dat' is '99852e026cee464dc60f61d617e05d3c2a71d8c7068d215dfb90ef06b541ef2a'
The SHA526 of 'randomdataDecoded.dat' is '0506f547b0f02251e5d05b77cce28bd46e2a6a5a7b2533a01dec6bc286307b0b'
Th hashes of the plaintext and decoded file should be the same


## AES Encryption and Decryption in CBC mode

The following script implements two functions for encrypting and decrypting data form a file in AES/CBC mode. 


In [76]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding


def encryptAESCBC(key,finput,foutput):
    # generate a new initialisation vector. This is done here to make sure it is renewed, each encryption pass
    iv = os.urandom(16)
    
    # create an encryptor object for the correct algorithm and mode
    encryptor = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    ).encryptor()

    # open the output file
    with open(foutput,"wb") as fo:
        # open the input file
        with open(finput,"rb") as fi:
            # read the contents of the input
            plaintext = fi.read()

            # Create padder, and padding to the input so that the length is correct
            padder = padding.PKCS7(128).padder()
            padded = padder.update(plaintext)
            padded += padder.finalize()

            # encrypt
            ciphertext = encryptor.update(padded) + encryptor.finalize()        

            # write the ciphertext to the output file
            fo.write(ciphertext)
            
    # return the IV, that is needed for decryption
    return iv


def decryptAESCBC(key,iv,finput,foutput):
    # initialize the algorithem and create a decryptor object
    decryptor = Cipher(
        algorithms.AES(key),
        modes.CBC(iv),
        backend=default_backend()
    ).decryptor()
    
    # open the output file
    with open(foutput,"wb") as fo:
        
        # open the input file
        with open(finput,"rb") as fi:
            
            # read the ciphertext
            ciphertext = fi.read()
        
            # decrypt
            decrypted = decryptor.update(ciphertext) + decryptor.finalize()
    
            # remove padding
            unpadder = padding.PKCS7(128).unpadder()
            unpadded = unpadder.update(decrypted)
            unpadded += unpadder.finalize()

            # write to the output
            fo.write(unpadded)

# This is our encryption key
key = bytes.fromhex("41b162773c263f60d4be893792b0c0119a3c1e3d16f881835e879487ca77827d")

# these are the filenames we're using
filenameInput = "randomdata.dat"
filenameEncrypted= "randomdataEncrypted.dat"
filenameDecoded = "randomdataDecoded.dat"

# create a random file of length 1000 bytes
createrandomfile(filenameInput,1000)

# encrypt the file and get the IV
iv = encryptAESCBC(key,filenameInput,filenameEncrypted)

# decrypt the file into a new file
decryptAESCBC(key,iv,filenameEncrypted,filenameDecoded)

# Dump the hashes of the files to check if everything went ok
for filename in [filenameInput, filenameEncrypted, filenameDecoded]:
    print("The SHA526 of '{}' is '{}'".format(filename,hashfile(filename,1)))
    
print("The hashes of the plaintext and decoded file should be the same")

The SHA526 of 'randomdata.dat' is '365cb47fd9659a4769f5c7098264e15f4ce6651df092a44a10efb8fe158de966'
The SHA526 of 'randomdataEncrypted.dat' is 'b52c41bc0678bf25d5eebb8a53fec818135b70de7dfe431e1b1f0b3c05a73925'
The SHA526 of 'randomdataDecoded.dat' is '365cb47fd9659a4769f5c7098264e15f4ce6651df092a44a10efb8fe158de966'
The hashes of the plaintext and decoded file should be the same


## Generating an RSA key pair

In [112]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization


def generate_private_key():
    # generate a private key
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048, 
        backend=default_backend() 
    )
    return private_key

def store_private_key(private_key,filename, password):
    # convert the private key to pem format
    private_key_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.TraditionalOpenSSL,
        #encryption_algorithm=serialization.NoEncryption()
        encryption_algorithm=serialization.BestAvailableEncryption(password)
    )

    # write the private key to file
    with open(filename,"wb") as f:
        f.write(private_key_pem)
    
def load_private_key(filename, password):
    # read the private key from file
    with open(filename,"rb") as f:
        # read the contents
        data = f.read()
        prvkey = serialization.load_pem_private_key(
            data,
            #password=None
            password=password
        )
        return prvkey
    
def store_public_key(public_key, filename):
    # convert the public key to pem
    public_key_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    # write the public key to file
    with open(pubkeyfile,"wb") as f:
        f.write(public_key_pem)

def load_public_key(filename):
    # read the public key from file
    with open(filename,"rb") as f:
        # read the contents of the file
        data = f.read()
        pubkey = serialization.load_pem_public_key(data)
    return pubkey

# define filename of the public key
pubkeyfile = "key.pub"
prvkeyfile = "key.prv"
password = b'correcthorsebatterystaple'
    
# generate a private key    
private_key = generate_private_key()

# store the private key
store_private_key(private_key,prvkeyfile,password)

# load the private key from file
private_key = load_private_key(prvkeyfile,password)

# derive public key from private key
public_key = private_key.public_key()

# store the public key to file
store_public_key(public_key,pubkeyfile)

# load the public_key from file
public_key = load_public_key(pubkeyfile)


## Encrypting and decrypting a message with RSA

In [118]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

def encryptRSA(pubkey, plaintext):
    ciphertext = pubkey.encrypt(
        plaintext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext

def decryptRSA(prvkey, ciphertext):
    plaintext = prvkey.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext

# message 
message = b'This some random message'

# load public key from file
public_key = load_public_key(pubkeyfile)

# encrypt the message
ciphertext = encryptRSA(public_key,message)

# dump the encrypted message
print("Encrypted message: ", ciphertext.hex())

# load the private key
private_key = load_private_key(prvkeyfile,password)

# decrypt the message
decoded = decryptRSA(private_key, ciphertext)

# dump the message
print("Decoded message: " + decoded.decode())



Encrypted message:  9e67446fecbd1bfbb1eca91c14fc66512a4c3a46bfa0ac03471fe872bf626fe74c45e082ccacf91b5497ec94952f498fec1307a8fce418e6dc2dc78857a594a79be75167ac70817501e5fb620afe3256f12acc0db2ce82799f62e7601d095ca25af29f05234adade6bd2517ba9a78cc4271af1e1386a03772f9290e38ffb058304d25dcb875ddb572a5274862511e69467427ced8a940108c2c514340abf00444b196c1b156006a83e486588156aaa260e643f1e3aeb476d2d6ef89f829d5ed461f14498786354deefc00ba1c71dc2b18b7257b3ed76120cef8e46a0f229211cf87c721014c54f5fe0b843ef0a533875d8a15d03bfe3dd04be62ec9e6681334c
Decoded message: This some random message


## Signing and veryfying with RSA


In [128]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.exceptions import InvalidSignature

def sign_message(private_key,message):
    signature = private_key.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),hashes.SHA256()
    )
    return signature

def verify_message(public_key, message, signature):
    try:
        public_key.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except InvalidSignature:
        return False
    

        # define a message
message = b"This is the message to be signed"

# load a private key
private_key = load_private_key(prvkeyfile,password)

# create a signature
signature = sign_message(private_key,message)

# load a public key
public_key = load_public_key(pubkeyfile)

# verify the signature of the message
result = verify_message(public_key, message, signature)

# print the result
print("Verification result (True is expected): ", result)

# verify the signature of another message
result = verify_message(public_key, b"another message", signature)

# print the result
print("Verification result (False is expected): ", result)




Verification result (True is expected):  True
Verification result (False is expected):  False


## TODO

Examples for the following cryptographic operations are to be included


  * Signing a message with an RSA private key
  * Verifying the signature of a message with an RSA public key
  
  